In [1]:
import time
import shutil
import os
import utils.run
import json


# Set Parameters

In [2]:
##### Global parameters
EXPERIMENT = 'supervised_pretraining_GPJ/'
experiment_name = 'CT_ewcmarpretraining_fullsupervision_ewcxtimestepsweep_batchsize32_episodes'
num_runs= 2 #Should either be an integer to commence all runs or list of specific runs 
           #(ie. num_runs=5 with run model 5 times, where num_runs=[5] with run only run 5)
gpu = 1 # gpu=1 use gpu, gpu=0 use cpu
stage = 9

In [3]:
N_TRIALS = 512 #TODO: Update
testing_batch_size = 32
DATAFILE1 =  'states_WSJ_correct_fullsupervision_ls.txt'
DATAFILE2 =  'states_WSJ_correct_fullsupervision_rs.txt'
PRETRAIN_EPOCHS = 25

### FILE VARIABLES
PRETRAIN_MODELNAME_ADITIONS = '_ewcmar' #extra suffix for the pretraining folder
TRAIN_MODELNAME_ADITIONS = '_ewcmarpretraining_ewcxtimestepsweep_batchsize32_episodes' #extra suffix for the training folder
### General
OVERWRITE = True #Default: False | whether any files should be overwritten
EXPERIMENT = 'supervised_pretraining_GPJ/' #Default: supervised_pretraining_<CORPUS> | experiment folder
#ROOT = '/mnt/c/files/research/projects/vid_game/data/'
ROOT = '/fs/clip-realspeech/projects/vid_game/data/' #Default: /fs/clip-realspeech/projects/vid_game/data | root folder on clip cluster
PARAMS_FOLDER = 'params' #Default: params | folder to store intermediate parameter files

### For Pretrain
PHONES_FILE = 'pretrain_phones' #Default: pretrain_phones | file containing phone list for pretraining
PRETRAIN_SEGMENTS_FILE = 'pretrain_segments' #Default: pretrain_segments | file containing segments list for pretraining
PRETRAIN_ALIGNMENTS_FILE = 'pretrain_alignments' #Default: pretrain_alignments | file containing alignments list for pretraining
WAVS_FOLDER = 'wavs_GPJ/' #Default: wavs_<CORPUS>/ | folder containing pretraining wavs

### For Validation
VALIDATION_SEGMENTS_FILE = 'validation_segments' #Default: validation_segments | file containing segments list for pretraining validation
VALIDATION_ALIGNMENTS_FILE = 'validation_alignments' #Default: validation_alignments | file containing alignments list for pretraining validation
VALIDATION_COPORA = ('GPJ',) #Default: Same as pretraining corpus | name of corpus to use for validation  **************************************


### For Train
GAME_WAVS_FOLDER = 'wavs_WSJ/' #Default: wavs_WSJ/ (was wavs for English) | Folder storing audio files for training
MODEL_FOLDER = 'models/' #Default: models/  | name of model output folder
OUT_FOLDER = 'exp/'  #Default: exp/ | name of experiment run folder
LOSS_OUT_FILE = 'loss' #Default: loss | name of loss ouptut file
RESULTS_FILE = 'results' #Default: results | name of result output file

### For Test
SIMPLE_STATE_TEST_FILE='test_states_simple' #Default: test_states_simple | NOTE CURRENTLY REQUIED Simple list of files to use during test results processing
ABX_WAVS_FOLDER = 'wavs_WSJ/'    #Default: wavs_WSJ/ (was wavs for English) | Folder storing audio samples for ABX

#########################################cd /fs/clip
### MODEL PARAMETERS
### General
BATCH_SIZE = 32 #Default: 32 | Size of batch to sample during training
UPDATES = 25 #Default: 25 | How often to print updates
CONV_SIZE = 10 #Deprecated

KERNEL = 5 #Default: 5 | Size of convolution kernel
STRIDE = 1 #Default: 1 | Stride of convolution
CONV1CHANNELS = 32 #Default: 32 | Number of channels in 1st convolution
CONV2CHANNELS = 32 #Default: 32 | Number of channels in 1st convolution
CONV3CHANNELS = 32 #Default: 32 | Number of channels in 1st convolution
MIDSIZE = 40 #Default: 40 | Number of nodes in linear layer
CONV_FREEZE = False #Default: False | Whether to freeze convolutional layers
CONV_FREEZE_LAYER = 0 #Default: 0 | Which convolutional layers should be frozen during training
FREEZE_LAYER_TIME = 0 #Default: 0 | At what episode convolutional layers should be frozen
LAYERS = (CONV1CHANNELS, CONV2CHANNELS, CONV3CHANNELS, MIDSIZE) # Does not need edited
SAMPLE_RATE = 16000 #Default: 16000 | Sample rate of acoustic input
WINDOW_SIZE = 0.2 #Default: 0.2 | Window size of acoustic token
SPEC_WINDOW_LENGTH = 400 #Default: 400 | Window length for spectrogram calculation
SPEC_WINDOW_HOP = 160 #Default: 160 | Spectrogram window hop size
N_FFT = 400 #Default: 400 | N_FFT parameter for spectrogram calculation

### For Pretrain
PRETRAIN_LR = 0.09 #Default" 0.09 | Learning rate during training
GAME_TYPE = 'convmovement' #Deprecated
GAME_MODE = 'oneshot' #Default: oneshot | Training game type
PRETRAIN_EPOCHS = 25 #Default: 25 | Number of epochs during pretraining

### For Validation
LOSS_TYPE = 'ewc' #Default: ewc | (or standard) Type of loss function to use during training
FISCHER_CORPUS='GPJ' #Default: Same as pretraining corpus | Corpus for which to calculate fischer coefficients
FISCHER_FILE = 'fischercoeffs' #Default: fischercoeffs | Name of fischer coefficient file
EWC_IMPORTANCE = [0, 0.00001, 0.00005, 0.0001, 0.0005, 0.001] #Default: ??? | EWC Importance Weighting Coefficient

### For Train
TRAIN_EPOCHS = [ 10, 25, 50, 100, 175, 250]
TRAIN_LR = 0.01 #Default: 0.05 | Learning rate during training
NUM_PHONES = 36   #Default: 36 (Japanese), 39 (English) | Number of phones in pretrained corpus


In [4]:
experiment_name = experiment_name+'_'+str(time.time())[1:10]
ROOT = ROOT + EXPERIMENT



## Parameters included in model names
### Pretrain
- PRETRAIN_MODELNAME
- PRETRAIN_LR
- KERNEL
- STRIDE
- BATCH_SIZE
- PRETRAIN_EPOCHS
- PRETRAIN_MODELNAME_ADITIONS

### Train
- TRAIN_MODELNAME
- GAME_MODE
- GAMMA
- EPS_DECAY
- TARGET_UPDATE
- WAITTIME
- KERNEL
- STRIDE
- CONV_FREEZE_LAYER
- FREEZE_LAYER_TIME
- LOSS_TYPE
- EWC_IMPORTANCE
- TRAIN_MODELNAME_ADITIONS
    

# Create Parameter Dictionaries

In [5]:
parameter_dicts = None
ps = {}
for key in dir():
    try:
        if key not in ['argparse','json','ps','save_params', 'f', 'args', 'parser', 'exit', 'get_ipython',
                       'parameter_dicts', 'In', 'Out', 'os', 'quit', 'shutil', 'utils', 'time'] and key[0] != '_':
            ps[key] = globals()[key]
    except:
         pass
parameter_dicts = utils.run.create_param_dictionaries(ps)
#print(len(parameter_dicts))
#parameter_dicts

# Save Parameters

In [6]:
# save parameters here
for ind in range(len(parameter_dicts)):
    parameter_dicts[ind]['param_name'] ='../params/'+experiment_name + '_' + str(ind) + '.params'
    parameter_dicts[ind]['model_id'] = str(ind)
    
    this_run = parameter_dicts[ind]
    #print(type(this_run))
    
    parameter_dicts[ind]['PRETRAIN_MODELNAME'] = 'pretrain_lr' + str(this_run['PRETRAIN_LR'])+ '_kernel' + str(this_run['KERNEL']) + \
                '_stride' + str(this_run['STRIDE']) + '_batchsize' + \
                str(this_run['BATCH_SIZE']) + '_epochs' + \
                str(this_run['PRETRAIN_EPOCHS'])+ this_run['PRETRAIN_MODELNAME_ADITIONS']
    
    parameter_dicts[ind]['TRAIN_MODELNAME'] = 'experiment_fullsupervision_' + \
                    '_kernel' + str(this_run['KERNEL']) + '_stride' + \
                str(this_run['STRIDE']) + '_lr'+str(TRAIN_LR) + '_freeze'+ \
                str(this_run['CONV_FREEZE_LAYER'])+'_freezetime' + str(this_run['FREEZE_LAYER_TIME']) + '_loss' + \
                str(this_run['LOSS_TYPE']) + str(this_run['EWC_IMPORTANCE']) + this_run['TRAIN_MODELNAME_ADITIONS'] + str(this_run['TRAIN_EPOCHS'])
    
    
    
    with open('../params/'+experiment_name + '_' + str(ind) + '.params', 'w') as f:
        json.dump(parameter_dicts[ind], f)

print('num_runs:', len(parameter_dicts))        
        
# archive code
if not os.path.exists('/fs/clip-realspeech/projects/vid_game/software/archive/'+experiment_name):
    os.makedirs('/fs/clip-realspeech/projects/vid_game/software/archive/'+experiment_name) 
shutil.copytree('/fs/clip-realspeech/projects/vid_game/software/VidGameRL', '/fs/clip-realspeech/projects/vid_game/software/archive/'+experiment_name+'1')



num_runs: 36


'/fs/clip-realspeech/projects/vid_game/software/archive/CT_ewcmarpretraining_fullsupervision_ewcxtimestepsweep_batchsize32_episodes_6490502031'

# Launch Jobs

In [7]:
# Launch jobs here 
for run in parameter_dicts:
    os.system(' '.join(['sbatch run_fullsupervision.sh', experiment_name, EXPERIMENT, run['param_name'], run['model_id'], str(gpu), str(stage), str(num_runs)]))
# EDITED: RUN_BATCH.SH GAME INSTEAD OF RUN_ACOUSTICGAME.SH
